# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [1]:
# Instalamos el modelo predictivo Prophet
!python -m pip install prophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [2]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [12]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2023-10-11'
start_date = '2016-01-01'
# Descargar el dataframe
cocacola_df = yf.download('KO',start_date, today).reset_index()

cocacola_df

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-01-04,$42.34,$42.50,$41.97,$42.40,$33.11,16084100
1,2016-01-05,$42.31,$42.63,$42.17,$42.55,$33.23,15781900
2,2016-01-06,$42.20,$42.51,$42.04,$42.32,$33.05,16245100
3,2016-01-07,$41.65,$42.11,$41.45,$41.62,$32.50,20563600
4,2016-01-08,$41.65,$42.12,$41.41,$41.51,$32.42,14645900
...,...,...,...,...,...,...,...
1951,2023-10-04,$54.89,$55.28,$54.71,$55.04,$55.04,12100500
1952,2023-10-05,$54.74,$54.95,$52.24,$52.38,$52.38,29878100
1953,2023-10-06,$52.04,$53.29,$51.55,$53.14,$53.14,30292000
1954,2023-10-09,$52.96,$53.15,$52.11,$52.88,$52.88,15704700


In [13]:
# TAREA: Validamos que no hay datos vacíos

cocacola_df.isna().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [14]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento

df = cocacola_df[['Date', 'Open']]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
new_names = {
    "Date": "ds",
    "Open": "y",
}

# Los renombramos con los nuevos nombres
df.rename(columns=new_names, inplace=True)
df['ds'] = df['ds'].dt.tz_localize(None)

In [15]:
df.tail()

,ds,y
1951,2023-10-04,$54.89
1952,2023-10-05,$54.74
1953,2023-10-06,$52.04
1954,2023-10-09,$52.96
1955,2023-10-10,$54.13


In [17]:
# Crear la gráfica del precio de apertura

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título
fig.update_layout(
    title_text="Serie temporal de Cocacola Precio de Apertura",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



Documentacion oficial de Prophet [GitHub](https://facebook.github.io/prophet/docs/quick_start.html)

In [18]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"

m = Prophet(seasonality_mode='multiplicative')

In [19]:
#TAREA: Entrena el modelo m con el dataframe de datos df

m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpoe44i_gt/qlnvjx8v.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpoe44i_gt/_uw6sql0.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=597', 'data', 'file=/tmp/tmpoe44i_gt/qlnvjx8v.json', 'init=/tmp/tmpoe44i_gt/_uw6sql0.json', 'output', 'file=/tmp/tmpoe44i_gt/prophet_model_lqqv7a9/prophet_model-20231011082436.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
08:24:36 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
08:24:39 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [20]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
2316,2024-10-05
2317,2024-10-06
2318,2024-10-07
2319,2024-10-08
2320,2024-10-09


In [21]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2016-01-04
1,2016-01-05
2,2016-01-06
3,2016-01-07
4,2016-01-08
...,...
2314,2024-10-03
2315,2024-10-04
2318,2024-10-07
2319,2024-10-08


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [24]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)

In [25]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2212,2024-10-03,$55.02,$32.78,$77.20
2213,2024-10-04,$55.01,$32.37,$77.48
2214,2024-10-07,$54.99,$32.24,$78.28
2215,2024-10-08,$55.10,$32.76,$77.76
2216,2024-10-09,$55.03,$31.53,$78.30


In [26]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

56.60700020223215

<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [27]:
plot_plotly(m, forecast)

In [28]:
plot_components_plotly(m, forecast)